# Optimal Lower Bounds In A Classical Traveling Salesman Problem

In order to get a better understanding of the TSP problem and why it is so computationally heavy, I came across [this paper](https://www.thechalkface.net/resources/Travelling_Salesman_England.pdf) that looks at 8 cities in England and generates upper and lower bounds for them. 

For all 8 cities, it looks at a complete graph of value 8. It generates an **upper bound** utilizing the nearest neighbor algorithm and it generates a lower bound by creating a **minimum spanning tree** utilizing **Kruskal's, Prim's or Boruvka's** algorithm. Since we already talked about the nearest neighbor algorithm in class, rather than looking at upper bounds, I am interested in seeing how lower bounds are generated and the interesting heuristic utilized by this paper to generate bounds *from every node*


### Figure 1: Distances from each UK city
<img src="./img_1.png"  width="600" height="600">
source: https://www.thechalkface.net/resources/Travelling_Salesman_England.pdf

## The Minimum Spanning Tree (MST) heuristic

- A spanning tree of a graph is a tree that spans G and includes every vertex and is a subgraph of G. Since it is a tree, it does not contain any cycles.
- It can be used to generate a lower bound for a TSP problem thus allowing us to come closer to finding an optimal solution. It also guarantees that the optimal tour MUST have a length greater than or equal to the MST generated bound. 
- A minimum spanning tree is a tree of graph G that's cumulative edge weights is less than the cumulative edge weights of other spanning trees since there can be multiple in G. 

- The first step of the MST algorithm is to add the smallest edge weight to the tree. There can be only 1 smallest edge weight. 



## Generating Minimum Spanning Tree

So how does this paper generate an MST from 'every' single city to get a lower bound and how does it transform the tree into a Hamiltonian cycle? 

### Figure 2: Upper and Lower Bounds Generated from Every UK City
<img src="./img_2.png"  width="600" height="600">
source: https://www.thechalkface.net/resources/Travelling_Salesman_England.pdf

- The way the paper does this is through the following: 
   - Removes the vertex $n_1$ from the graph G and all adjacent edges.  
   - Creates an MST of the graph $(G - n_1)$
   - Finds the 2 smallest edge weights that come out of the removed vertex $n_1$ and reappends those to the graph along with the node itself
   


In [2]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import networkx as nx
import random

### Visualising The Graph
We will now attempt to visualise this complete graph, input the distance data and then iterate over the graph, generating spanning trees from all 8 cities and calculating the total distances from each city. This should gives us the same lower bounds found in the table above. 

In [3]:
# Setup the UK Cities Graph with labels
G1 = nx.complete_graph(8)

labels = {0:'London', 1:'Birmingham', 2:'Leeds', 3:'Sheffield', 4:'Bradford', 5:'Liverpool', 6:'Manchester', 7:'Bristol'}
G2 = nx.relabel_nodes(G1, labels)

# Add seed 30 in order to get the same graph figure everytime
my_pos = nx.spring_layout(G2, seed = 30)

# Adding weight attributes (distance) to the cities based on Figure 1
distance_matrix = [118, 200, 167, 207, 211, 199, 
                   119, 124, 90, 129, 98, 86, 88, 
                   40, 10, 72, 44, 208, 48, 78, 78, 
                   180, 67, 39, 165, 34, 181, 168]
 
count = 0
for e in G2.edges():
    G2.edges[e]['distance'] = distance_matrix[count]
    count += 1
    if (count == len(G1.edges())):
        break
        
nx.draw(G2, pos = my_pos, with_labels = True)

Now that we have our complete graph, we will utilize NetworkX's built in Minimum Spanning Tree function `nx.minimum_spanning_tree(G2, weight = 'weight', algorithm='kruskal')`to generate MSTs from every individual node. 

- The documentation over [here](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.tree.mst.minimum_spanning_tree.html) provides us with a better understanding of how this function works. 
- Essentially, we pass it a graph, provide a key for what values it should use for edge weights and lastly pass it one of three algorithms (Prim's, Kruskal's or Boruvka's) to use to calculate the MST. 
- Since my graph is called G2, I pass in G2. Since my edge weights are labelled as 'distances', I pass 'distances' for the second parameter. Lastly, I use Kruskal's as it is the default. 



In [5]:
# Have a copy of G2 before each individual vertex is removed so its min 2 edges can be added back later
Curr = G2.copy()

# creating 2x4 table to show all 8 MSTs side by side 
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(9, 9))
ax = axes.flatten()
graphs = []

# find total distance by summing all the edges of the given graph/tree
def total_dist_sum(G):
    total_dist = 0
    for e in T.edges():
        total_dist += T.edges[e]['distance']
    return total_dist 

# find the two smallest values in an array (used to reappend to the MST)
def find_2min(arr):
    arr2 = []
    arr2.append(min(arr))
    arr.remove(min(arr))
    arr2.append(min(arr))
    return arr2

min_2_arr = []
# Create a graph 'G2' that has 1 vertex and all corresponding edges removed
for i in range(0, 8):
    min_2_arr = []
    two_locations = []
    arr3 = []
    total_dist2 = 0
    
#   we remove the n_1 vertex (so first we remove London), then apply the MST heuristic and find the lower bound accordingly
#   we then proceed to do this for all cities (remove the node, apply MST heuristic, calculate total distance) 
    G2.remove_node(labels[i])
    
#   create an MST without London, using Kruskal's algorithm 
    T = nx.minimum_spanning_tree(G2, weight = 'distance', algorithm='kruskal')
    
#   find total value of this MST
    total_value_mst = total_dist_sum(T)
    
#   looking at the edges of Curr since it still has the original vertex
    for e in Curr.edges():
        if (e[0] == labels[i] or e[1] == labels[i]):
            min_2_arr.append(Curr.edges[e]['distance'])
            
    arr3 = find_2min(min_2_arr)
    total_dist2 = total_value_mst + arr3[0] + arr3[1]        
    G2 = Curr.copy()
    print('Lower bound starting from', labels[i], 'is:', total_dist2)
    T.add_node(labels[i])
    graphs.append(T)
    

#   display all 8 MST figures 
for i in range(0, 8):
    nx.draw_networkx(graphs[i], pos = my_pos, node_size=18, font_size = 9, ax=ax[i])
    ax[i].set_axis_off()   

   

Lower bound starting from London is: 534
Lower bound starting from Birmingham is: 581
Lower bound starting from Leeds is: 463
Lower bound starting from Sheffield is: 463
Lower bound starting from Bradford is: 459
Lower bound starting from Liverpool is: 482
Lower bound starting from Manchester is: 486
Lower bound starting from Bristol is: 534


### We see that the lower bound values from each city are identical to the ones in the table
The purpose of the code above was the following:
- Utilize the networkx library to visualise a complete graph of $n=8$ and assign weights and labels to each edge/vertex based on real world city data. 
- Remove the vertex that we're starting from, generate an MST, find the 2 smallest distances from the removed vertex and add that to the total MST weight. This gives us the total distance for a Hamiltonian Cycle from a specific city that also serves as a lower bound. 
- Display all the MSTs that were created 